In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from scipy.ndimage import gaussian_filter, laplace

In [ ]:
# Define paths and parameters
base_dir = "D:/DATASET/CNN/ballooning/train-copy"
val_dir = "D:/DATASET/CNN/ballooning/val" 
target_dir = "D:/DATASET/CNN/ballooning/balanced_train"
batch_size = 32
target_size = (185, 932)  # New target size for cropping

In [ ]:
# Function to save balanced dataset
def save_balanced_dataset(source_dir, target_dir):
    # Create target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        os.makedirs(os.path.join(target_dir, '0'))
        os.makedirs(os.path.join(target_dir, '1'))

    # Load dataset
    data_gen = ImageDataGenerator(rescale=1./255)
    generator = data_gen.flow_from_directory(
        source_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        save_to_dir=target_dir,
        save_prefix='aug_',
        save_format='jpeg'
    )

    # Balance classes
    for _ in range(2000):  # Number of batches to generate
        images, labels = next(generator)

In [ ]:
# Load and preprocess data
def preprocess_data():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_generator = train_datagen.flow_from_directory(
        target_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary'
    )
    return train_generator

In [ ]:
# Load model and add custom layers
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Train model
def train_model(model, train_generator):
    history = model.fit(train_generator, epochs=5, validation_data=validation_generator)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    return model

In [ ]:
# Predict using model
def predict_image(model, img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, color_mode='rgb')
    img = image.img_to_array(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img = gaussian_filter(img, sigma=1)  # Apply Gaussian filter
    img = laplace(img)  # Apply Laplacian filter to highlight features
    # Crop and predict for each patch
    predictions = []
    for y in range(0, img.shape[0], target_size[1]):  # Loop through height
        crop = img[y:y+target_size[1], :]
        crop = cv2.resize(crop, (224, 224))
        crop = np.expand_dims(crop, axis=-1)
        crop = np.repeat(crop, 3, axis=-1)  # Make 3 channel for ResNet input
        crop = np.expand_dims(crop, axis=0)
        prediction = model.predict(crop)[0]
        predictions.append(prediction)
    average_score = np.mean(predictions)
    plt.imshow(img, cmap='gray')
    plt.title(f'Processed Image - Average Score: {average_score:.2f}')
    plt.show()
    return average_score

In [ ]:
model = build_model()

In [ ]:
train_generator = preprocess_data()

In [ ]:
model = train_model(model, train_generator)

In [ ]:
img_path = "D:/DATASET/CNN/ballooning/val/1/97_5_43.png"

In [ ]:
average_score = predict_image(model, img_path)